In [ ]:
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

In [16]:
import torch
from torchvision.transforms import Resize
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure, LearnedPerceptualImagePatchSimilarity
import utils
from models import UNet, SkipNet
from denoisers import DIP, DIP_MWV, DIP_TV, DDIP, GaussianBlur, SelfDIP

In [ ]:
# clean = utils.load_celeba(1)
# clean = utils.load_images('./data/CBSD68/', Resize((256, 256)))
# clean = utils.load_images('./data/Set14/', Resize((256, 256)))
clean = utils.load_image("./data/set5/butterfly.png")
noisy = torch.stack([utils.get_noisy_image(img, 10) for img in clean])

utils.plot_row([clean[0], noisy[0]])

In [ ]:
skipnet1 = SkipNet(3, [8, 16, 32, 64, 128], [0, 0, 0, 4, 4])
skipnet2 = SkipNet(3, [128, 128, 128, 128, 128], [4, 4, 4, 4, 4])
unet = UNet(hidden_ch=8, n_layers=5)

denoisers = [DIP(skipnet1), DIP_MWV(skipnet1), DDIP(skipnet1)]

results = {}
for denoiser in denoisers:
    outputs = []
    for i, (x_hat, x) in enumerate(zip(noisy, clean)):
        options = {
            "mode": "local",
            "metrics": ["psnr", "ssim", "lpips"],
            "config": {
                "project": "zero-shot-das-denoising",
                "entity": "jmaen-team", 
                "group": "xxx",
                "dataset": "xxx",
                "id": i,
            }
        }
        output = denoiser.denoise(x_hat.unsqueeze(0), x.unsqueeze(0), options)
        output = output.detach().cpu()
        outputs.append(output)

    results[str(denoiser)] = torch.cat(outputs)

In [ ]:
for result in zip(noisy, *results.values(), clean):
    utils.plot_row(result, ["Noisy", *list(results.keys()), "Clean"])

In [ ]:
metrics = {
    "PSNR": PeakSignalNoiseRatio(reduction=None, data_range=1, dim=[1, 2, 3]),
    "SSIM": StructuralSimilarityIndexMeasure(reduction=None, data_range=1),
    "LPIPS": LearnedPerceptualImagePatchSimilarity(),
}

for key, metric in metrics.items():
	print(key)
	print("----------")
	print(f"Noisy:".ljust(30), f"{metric(noisy, clean).mean()}")
	for name, xs in results.items():
		s = metric(xs, clean)
		print(f"{name}:".ljust(30), f"{s.mean()}\t{s.tolist()}")
	print("\n")